In [6]:
import numpy as np
from filterpy.kalman import KalmanFilter
import pandas as pd

In [8]:
for d in np.arange(1,6):
    print(f"d = {d}")
    kf = KalmanFilter(dim_x=d, dim_z = 3)

    # Initialize all of the matrices

d = 1
d = 2
d = 3
d = 4
d = 5
